## Generate synthetic data for DPO

This notebook creates a synthetic dataset of classified ads and scores them in pairs.

In [1]:
#%pip install --upgrade --quiet google-genai

In [4]:
MODEL_ID = "models/gemini-2.0-flash"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env");
assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
       "Please specify the GEMINI_API_KEY access token in keys.env file"

In [5]:
from google import genai
from google.genai import types

client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))

## Create an ad for an item

In [6]:
import random
def create_classified_ad(item: str, price: str) -> str:
    prompt = f"""
    You are writing a classified ad for used items in a neighborhood online forum. Write a one paragraph description for
    a {item} priced at {price}
    """
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=[prompt],
        config=types.GenerateContentConfig(
            max_output_tokens=500,
            temperature=random.uniform(0.2, 0.9),
        )
    )
    return response.text

create_classified_ad("3-year old Specialized bike", "$300")

'Lightly used, 3-year-old Specialized mountain bike in excellent condition. Perfect for hitting the trails or cruising around town. Features a sturdy frame, reliable brakes, and smooth shifting gears. Regularly maintained and always stored indoors. Asking $300. Serious inquiries only, please. Contact [Your Name] at [Your Phone Number or Email Address] to schedule a viewing.\n'

In [9]:
example_ad = create_classified_ad("3-year old Specialized bike", "$300")
example_ad

'Lightly used, 3-year-old Specialized hybrid bike in excellent condition. Perfect for commuting, recreational rides, or exploring bike trails. Features a comfortable saddle, smooth-shifting gears, and reliable brakes. Well-maintained and ready to ride. Asking $300. Local pickup only. Serious inquiries only, please.\n'

## Score an ad

In [33]:
from dataclasses import dataclass

@dataclass
class AdScore:
    clarity: int
    audience_targeting: int
    readability: int
    contact_info: int
    
    def total_score(self) -> int:
        return self.clarity + self.audience_targeting + self.readability + self.contact_info

def score_ad(ad: str) -> AdScore:
    prompt = f"""
    You are a professor of advertising at a business school. Score the following classified ad based on the following criteria:
    * Is it clear what's being sold? Age, brand, price, and condition are important. (10 points)
    * Does it target the most relevant audience for the item? Is it persuasive to that audience? (10 points)
    * Is it concise and easy to read? (10 points)
    * Does it include contact information? Ideally, the ad specifies the preferred means of communication. (10 points)
    
    Ad:
    {ad}
    """
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=[prompt],
        config=types.GenerateContentConfig(
            temperature=0.1,
            response_mime_type='application/json',
            response_schema=AdScore
        )
    )
    
    return response.parsed

In [34]:
example_score = score_ad(example_ad)
example_score

AdScore(clarity=9, audience_targeting=8, readability=9, contact_info=2)

In [35]:
example_score.total_score()

28

## Generate data

In [38]:
def create_preference_example(item: str, price: str) -> dict:
    ad1 = create_classified_ad(item, price)
    ad2 = create_classified_ad(item, price)
    score1 = score_ad(ad1).total_score()
    score2 = score_ad(ad2).total_score()
    
    preference_example = {
        "prompt": f"Selling {item} at {price}"
    }
    
    if score1 > score2:
        preference_example['chosen'] = ad1
        preference_example['rejected'] = ad2
    else:
        preference_example['chosen'] = ad2
        preference_example['rejected'] = ad1
    
    return preference_example

create_preference_example("3-year old Specialized bike", "$300")

{'prompt': 'Selling 3-year old Specialized bike at $300',
 'chosen': 'Lightly used, 3-year-old Specialized hybrid bike in excellent condition. Perfect for commuting, recreational rides, or exploring local trails. Features a comfortable saddle, smooth-shifting gears, and responsive brakes. Well-maintained and ready to ride. Asking $300. Serious inquiries only, please contact [Your Name] at [Your Contact Information] to schedule a viewing.\n',
 'rejected': 'Get ready to ride with this gently used, 3-year-old Specialized bike! Perfect for cruising around the neighborhood or hitting local bike paths, this reliable ride is in great condition and ready for a new owner. Asking $300, cash only. Serious inquiries only, please. Contact [Your Contact Information] to arrange a viewing.\n'}

In [46]:
import json

items_for_sale = [
    ("3-year old Specialized road bike", "$300"),
    ("Amazing Spider-Man 361", "$200"),
    ("Like-new copy of Chinua Achebe's Things Fall Apart", "$5"),
    ("6-piece Le Creuset cookware set in good condition", "$800"),
]

def write_jsonl(num_examples: int, filename: str):
    examples = []
    for iter in range(num_examples):
        print(iter, end=" ... ")
        item, price = random.choice(items_for_sale)
        example = create_preference_example(item, price)
        examples.append(example)
    
    with open(filename, "w") as ofp:
        for example in examples:
            json.dump(example, ofp)
            ofp.write('\n')
            
    return examples, filename

In [49]:
examples, filename = write_jsonl(2, "ad_preference_dataset.jsonl")

0 ... 1 ... 

In [50]:
len(examples)

2

In [51]:
!head *.jsonl

{"prompt": "Selling 6-piece Le Creuset cookware set in good condition at $800", "chosen": "Elevate your cooking game with this beautiful 6-piece Le Creuset cookware set! This collection includes a Dutch oven, saucepan, skillet, and lids, all in the classic flame color. Gently used and well-maintained, these pieces are in good condition and ready to provide years of reliable service in your kitchen. A fantastic opportunity to own premium, enameled cast iron cookware at a fraction of the retail price. Asking $800 for the entire set. Serious inquiries only, please.\n", "rejected": "Elevate your cooking game with this beautiful 6-piece Le Creuset cookware set! This collection includes a Dutch oven, saucepan, skillet, and three lids, all in vibrant flame orange. While gently used, each piece is in excellent condition with minimal wear and tear, offering the legendary Le Creuset heat retention and even cooking you know and love. Retailing for over $1500 new, this set is yours for the taking 

## Do it

In [52]:
write_jsonl(100, "ad_preference_dataset.jsonl");

0 ... 1 ... 2 ... 3 ... 4 ... 5 ... 6 ... 7 ... 8 ... 9 ... 10 ... 11 ... 12 ... 13 ... 14 ... 15 ... 16 ... 17 ... 18 ... 19 ... 20 ... 21 ... 22 ... 23 ... 24 ... 25 ... 26 ... 27 ... 28 ... 29 ... 30 ... 31 ... 32 ... 33 ... 34 ... 35 ... 36 ... 37 ... 38 ... 39 ... 40 ... 41 ... 42 ... 43 ... 44 ... 45 ... 46 ... 47 ... 48 ... 49 ... 50 ... 51 ... 52 ... 53 ... 54 ... 55 ... 56 ... 57 ... 58 ... 59 ... 60 ... 61 ... 62 ... 63 ... 64 ... 65 ... 66 ... 67 ... 68 ... 69 ... 70 ... 71 ... 72 ... 73 ... 74 ... 75 ... 76 ... 77 ... 78 ... 79 ... 80 ... 81 ... 82 ... 83 ... 84 ... 85 ... 86 ... 87 ... 88 ... 89 ... 90 ... 91 ... 92 ... 93 ... 94 ... 95 ... 96 ... 97 ... 98 ... 99 ... 

In [53]:
!wc -l *.jsonl

100 ad_preference_dataset.jsonl
